# Data manipulation

In [1]:
import pandas as pd

In [2]:
with open('../rucoref_29/Tokens.txt') as f:
    tokens = f.read()
with open('../rucoref_29/Groups.txt') as f:
    coreference = f.read()
    
#prons = 'он она оно они его её их мой'.split()

In [31]:
tokens = pd.read_csv('../rucoref_29/Tokens.txt', sep='\t')
tokens = tokens.rename(index=str, columns={"shift": "shift_o", "length": "length_o"})
tokens['ID'] = ['']*len(tokens)
#for token_id, token in tokens.iterrows():
#    tokens.loc[token_id, 'ID'] = str(token['doc_id'])+'-'+str(token['shift_o'])
tokens['ID'] = tokens['doc_id'].apply(str).str.cat(['-'+str(w) for w in tokens['shift_o'].values.tolist()])
tokens.head()

,doc_id,shift_o,length_o,token,lemma,gram,ID
0,1,0,2,Во,во,Sp-a,1-0
1,1,3,5,время,время,Ncnsan,1-3
2,1,9,5,своих,свой,P---pga,1-9
3,1,15,8,прогулок,прогулка,Ncfpgn,1-15
4,1,24,1,в,в,Sp-l,1-24


In [36]:
tokens['ID'].is_unique

True

In [32]:
coreference = pd.read_csv('../rucoref_29/Groups.txt', sep='\t')

coreference.head()

,doc_id,variant,group_id,chain_id,link,shift,length,content,tk_shifts,attributes,head,hd_shifts
0,1,1,407840,1070,0,9,5,своих,9,ref:def|str:refl|type:coref,NaN,NaN
1,1,1,407839,1070,407840,47,1,я,47,ref:def|str:pron|type:coref,NaN,NaN
2,1,1,407842,1069,0,69,13,одинокую дачу,"69,78",ref:def|str:noun|type:coref,дачу,78
3,1,1,407841,1069,407842,118,9,этой даче,"118,123",ref:def|str:noun|type:coref,даче,123
4,1,1,407843,1069,407841,166,3,она,166,ref:def|str:pron|type:coref,NaN,NaN


In [90]:
def add_coref_to_token(token_id, tokens_with_coref, coref_info, cols):
    #if pd.isnull(tokens_with_coref.loc[tokens_with_coref['ID'] == token_id]['content'].all()):
    for col in cols:
        tokens_with_coref.loc[tokens_with_coref['ID'] == token_id, col] = coref_info[col]
    if len(str(coref_info['attributes']).split('|')) > 1:
        if coref_info['attributes'].split('|')[2].split(':')[1] == 'anaph':
            tokens_with_coref.loc[tokens_with_coref['ID'] == token_id, col] = 1
        else:
            tokens_with_coref.loc[tokens_with_coref['ID'] == token_id, col] = 0
    #else:
    '''    print(tokens_with_coref.loc[tokens_with_coref['ID'] == token_id])
        tokens_with_coref.loc[len(tokens_with_coref)+1] = tokens_with_coref.loc[tokens_with_coref['ID'] == token_id]
        for col in cols:
            tokens_with_coref.loc[tokens_with_coref['ID'] == token_id, col] = coref_info[col]
        if len(str(coref_info['attributes']).split('|')) > 1:
            if coref_info['attributes'].split('|')[2].split(':')[1] == 'anaph':
                tokens_with_coref.loc[tokens_with_coref['ID'] == token_id, col] = 1
            else:
                tokens_with_coref.loc[tokens_with_coref['ID'] == token_id, col] = 0
        '''
    return tokens_with_coref

In [91]:
from tqdm import tqdm_notebook as tqdm

In [92]:
cor_list = coreference.values.tolist()
tokens_with_coref = tokens.copy()
cols = coreference.columns.values.tolist()
cols.pop(0)

tokens_with_coref = tokens_with_coref.reindex(columns = tokens.columns.values.tolist() + cols + ['anaph'])
#tokens_with_coref.head()

In [93]:
c = 0
for coref_id, coref in tqdm(coreference.iterrows()):
    c+=1
    if not pd.isnull(coref['head']):
        token_id = str(coref['doc_id']) + '-' + str(coref['hd_shifts'])
    else:
        token_id = str(coref['doc_id']) + '-' + str(coref['shift'])
    tokens_with_coref = add_coref_to_token(token_id, tokens_with_coref, coref, cols)

#len(tokens_with_coref['content'].count)

TypeError: object of type 'method' has no len()

In [100]:
tokens_with_coref[:20].fillna('-')

,doc_id,shift_o,length_o,token,lemma,gram,ID,variant,group_id,chain_id,link,shift,length,content,tk_shifts,attributes,head,hd_shifts,anaph
0,1,0,2,Во,во,Sp-a,1-0,-,-,-,-,-,-,-,-,-,-,-,-
1,1,3,5,время,время,Ncnsan,1-3,-,-,-,-,-,-,-,-,-,-,-,-
2,1,9,5,своих,свой,P---pga,1-9,1,407840,1070,0,9,5,своих,9,ref:def|str:refl|type:coref,-,0,-
3,1,15,8,прогулок,прогулка,Ncfpgn,1-15,-,-,-,-,-,-,-,-,-,-,-,-
4,1,24,1,в,в,Sp-l,1-24,-,-,-,-,-,-,-,-,-,-,-,-
5,1,26,12,окрестностях,окрестность,Ncfpln,1-26,-,-,-,-,-,-,-,-,-,-,-,-
6,1,39,7,Симеиза,симеиза,Ncmsgn,1-39,-,-,-,-,-,-,-,-,-,-,-,-
7,1,47,1,я,я,P-1-snn,1-47,1,407839,1070,407840,47,1,я,47,ref:def|str:pron|type:coref,-,0,-
8,1,49,7,обратил,обратить,Vmis-sma-p,1-49,-,-,-,-,-,-,-,-,-,-,-,-
9,1,57,8,внимание,внимание,Ncnsan,1-57,-,-,-,-,-,-,-,-,-,-,-,-


In [101]:
tokens_with_coref.to_csv('tokens_with_coref.csv')

In [ ]:
cor_list = coreference.values.tolist()
tokens_with_coref = tokens.copy()
cols = coreference.columns.values.tolist()
cols.pop(0)

tokens_with_coref = tokens_with_coref.reindex(columns = tokens.columns.values.tolist() + cols + ['anaph'])

for i in range(len(tokens)):
    if cor_list[0][10] == '-':
        if tokens['shift_o'][i] == cor_list[0][5]:
            for col_n in range(len(cols)):
                tokens_with_coref.iloc[i, col_n+6] = cor_list[0][col_n+1]
            #print('added:')
            if len(str(cor_list[0][9]).split('|')) > 1:
                if cor_list[0][9].split('|')[2].split(':')[1] == 'anaph':
                    tokens_with_coref.iloc[i, 17] = 1
            if cor_list[0][5] != cor_list[1][5]:
                cor_list.pop(0)
            else:
                row = tokens.iloc[i].values.tolist() + cor_list[1][1:]
                tokens_with_coref.append(row)
                cor_list.pop(0)
                cor_list.pop(0)
    #print(tokens['shift_o'][i] == cor_list[0][5], cor_list[0][5], cor_list[0][7])

False 9 своих
False 9 своих
False 47 я
False 47 я
False 47 я
False 47 я
False 47 я
False 69 одинокую дачу
False 69 одинокую дачу
False 69 одинокую дачу
False 69 одинокую дачу
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 118 этой даче
False 166 она
False 166 она
False 166 она
False 166 она
False 166 она
False 166 она
False 166 она
False 166 она
False 166 она
False 184 высоким забором
False 184 высоким забором
False 184 высоким забором
False 203 единственной низкой калиткой
False 203 единственной низкой калиткой
False 203 единственной низкой калиткой
False 203 единственной низкой калиткой
False 233 которая
False 233 которая
False 233 которая
False 233 которая
False 316 забором
False 316 забором
False 316 забором
False 316 забором
False 316 забором
False 316 забором
False 316 забором
False 316 забором
False 316 забором
False 316 забором
False 316 заб

/Users/Ksenia/anaconda/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


False 1290 себя
False 1290 себя
False 1290 себя
False 1290 себя
False 1290 себя
False 1290 себя
False 1290 себя
False 1326 они
False 1326 они
False 1326 они
False 1326 они
False 1326 они
False 1326 они
False 1326 они
False 1355 Они
False 1355 Они
False 1355 Они
False 1355 Они
False 1355 Они
False 1355 Они
False 1355 Они
False 1388 мой
False 1388 мой
False 1388 мой
False 1388 мой
False 1388 мой
False 1388 мой
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1438 мне
False 1498 их
False 1498 их
False 1498 их
False 1498 их
False 1498 их
False 1498 их
False 1498 их
False 1498 их
False 1498 их
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1570 ты
False 1573 мужчина
False 1582 ты
False 1582 т

In [6]:
#print(tokens_with_coref.iloc[1483:1500, 0:6])

#print(cor_list[0:3])
print(len(coreference), len(tokens))
tokens_with_coref.count()

16557 156636


doc_id        156636
shift_o       156636
length_o      156636
token         156636
lemma         156636
gram          156205
variant         8373
group_id        8373
chain_id        8373
link            8373
shift           8373
length          8373
content         8373
tk_shifts       8373
attributes      8367
head            2211
hd_shifts       2211
anaph           1371
dtype: int64

In [7]:
tokens_with_coref['anaph'] = tokens_with_coref['anaph'].fillna(0)
tokens_with_coref = tokens_with_coref.fillna('-')

sum(tokens_with_coref['anaph'])

1371.0

In [24]:
from string import punctuation


not_found = []
anaphora = pd.DataFrame({#'chain_id':[],
                        "target":[],"t_gend":[],
                         "t_count":[],"cand":[],"c_gend":[],
                         "c_count":[],'c_pow':[],'dist':[],'answ':[]})
for i, row in tokens_with_coref.iterrows():
    counter = 0
    #print(row.values)
    #print(row['anaph'], row['link'])
    if row['anaph'] == 1 and row['link'] != 0:
        #chain_id = row['chain_id']
        shift = row['shift']
        target_link = row['link']
        target = row['lemma']
        if len(row['gram']) > 4:
            target_gend = row['gram'][4]
            target_count = row['gram'][3]
        else:
            target_gend = '-'
            target_count = '-'
        
        number = 0
        found_answ = False
        while number <= 20:
            counter += 1
            
            if tokens_with_coref.iloc[int(i)-counter-1]['lemma'] not in punctuation:
                i = int(i)
                if tokens_with_coref.iloc[int(i)-counter-1]['head'] == '-':
                    candidate = tokens_with_coref.iloc[i-counter-1]['lemma']
                else:
                    candidate = 
                
                if len(tokens_with_coref.iloc[i-counter-1]['gram']) > 4:
                    cand_gend = tokens_with_coref.iloc[i-counter-1]['gram'][4]
                    cand_count = tokens_with_coref.iloc[i-counter-1]['gram'][3]
                else:
                    cand_gend = '-'
                    cand_count = '-'
                cand_pow = tokens_with_coref.iloc[i-counter-1]['gram'][0]
                dist = counter + 1
                number += 1
                if tokens_with_coref.iloc[i-counter-1]['group_id'] == target_link:
                    answ = 1
                    found_answ = True
                else:
                    answ = 0
                #print([target, target_gend, target_count,
                 #               candidate, cand_gend, cand_count, cand_pow, dist, answ])
                anaphora.loc[len(anaphora)+1] = [answ, cand_count, cand_gend, cand_pow, candidate,
                                                 dist, target_count, target_gend, target]

        if not found_answ:
            #print('WARNING: answ_not_found for ', target, shift, i)
            not_found.append([target, shift, i])
    #anaphora.append()
    
    #if i == '200':
    #    break
print(len(not_found))
anaphora.head(40)

174


,answ,c_count,c_gend,c_pow,cand,dist,t_count,t_gend,target
1,0.0,p,g,N,скала,3.0,-,p,они
2,0.0,m,p,A,желтоватый,4.0,-,p,они
3,0.0,p,n,N,уступ,5.0,-,p,они
4,1.0,m,p,A,голый,6.0,-,p,они
5,0.0,s,g,N,дача,8.0,-,p,они
6,0.0,-,-,S,кругом,9.0,-,p,они
7,0.0,s,i,N,забор,11.0,-,p,они
8,0.0,-,-,S,над,12.0,-,p,они
9,0.0,s,-,V,виднеться,13.0,-,p,они
10,0.0,-,-,Q,не,14.0,-,p,они


In [41]:
#from statistics import mean
#anaphora = anaphora.rename(columns = {'dist':'t_gend', 't_gend':'dist'})
new_an = anaphora[["t_gend",
                 "t_count", "c_gend",
                 "c_count",'c_pow','dist','answ']]
print(new_an.head())

  t_gend t_count c_gend c_count c_pow  dist  answ
1      p       -      g       p     N   3.0   0.0
2      p       -      p       m     A   4.0   0.0
3      p       -      n       p     N   5.0   0.0
4      p       -      p       m     A   6.0   1.0
5      p       -      g       s     N   8.0   0.0


# Testing simple models

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [42]:
test, train = train_test_split(new_an, test_size=0.3, random_state=14)

In [43]:
lr = LogisticRegression()#solver='sag', multi_class='multinomial', n_jobs=-1, max_iter=100)

params = [{'penalty': ['l1','l2'], 'class_weight':[None,'balanced'], 'max_iter':[100, 150, 250]}]
clf = GridSearchCV(lr, params)

rf = RandomForestClassifier()
rf.fit(train.iloc[1:], train['answ'])
print(classification_report(tes['answ'], lr.predict(test.iloc[1:])))
# print(clf.best_params_, clf.best_score_)

ValueError: could not convert string to float: 'N'